In [2]:
import deepxde as dde
import numpy as np
import tensorflow as tf

# Define the ODE system: y''(x) + y(x) = 0
def ode_system(x, y):
    dy_xx = dde.grad.hessian(y, x)
    return dy_xx + y

# Define the domain
geom = dde.geometry.Interval(0, np.pi)

# Define the boundary conditions
def boundary_left(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

def boundary_right(x, on_boundary):
    return on_boundary and np.isclose(x[0], np.pi)

bc_left = dde.DirichletBC(geom, lambda x: 0, boundary_left)
bc_right = dde.DirichletBC(geom, lambda x: 0, boundary_right)

# Create the data object
data = dde.data.PDE(
    geom,
    ode_system,
    [bc_left, bc_right],
    num_domain=10,
    num_boundary=2,
)

# Define the neural network
net = dde.maps.FNN([1] + [20] * 3 + [1], "tanh", "Glorot normal")

# Define a custom loss function
def custom_loss(y_true, y_pred):
    # PDE loss (DeepXDE will handle this part automatically)
    pde_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    
    # Custom regularization term (e.g., penalizing large values of y_pred)
    regularization = tf.reduce_mean(tf.square(y_pred))
    
    # Combine the losses
    total_loss = pde_loss + 0.1 * regularization  # Here, 0.1 is the weight for the regularization term
    return total_loss

# Define the model
model = dde.Model(data, net)

# Compile the model with the custom loss function
model.compile("adam", lr=1e-3, loss=custom_loss)

# Train the model
losshistory, train_state = model.train(epochs=10000)

# Plot the results
dde.saveplot(losshistory, train_state, issave=True, isplot=True)


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.



Instructions for updating:
non-resource variables are not supported in the long term
Compiling model...
Building feed-forward neural network...

'build' took 0.191358 s



c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(



'compile' took 0.440432 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [4.57e-02, 0.00e+00, 9.56e-02]    [4.57e-02, 0.00e+00, 9.56e-02]    []  
1000      [3.05e-06, 3.39e-08, 4.80e-08]    [3.05e-06, 3.39e-08, 4.80e-08]    []  
2000      [9.90e-08, 3.26e-10, 4.42e-11]    [9.90e-08, 3.26e-10, 4.42e-11]    []  
3000      [5.96e-08, 1.34e-11, 1.66e-10]    [5.96e-08, 1.34e-11, 1.66e-10]    []  
4000      [3.86e-05, 2.60e-06, 1.08e-04]    [3.86e-05, 2.60e-06, 1.08e-04]    []  
5000      [5.27e-08, 8.00e-10, 2.06e-08]    [5.27e-08, 8.00e-10, 2.06e-08]    []  
6000      [3.86e-08, 4.89e-11, 4.73e-10]    [3.86e-08, 4.89e-11, 4.73e-10]    []  
7000      [8.46e-06, 5.68e-07, 2.39e-05]    [8.46e-06, 5.68e-07, 2.39e-05]    []  
8000      [3.05e-08, 7.58e-12, 3.20e-11]    [3.05e-08, 7.58e-12, 3.20e-11]    []  


KeyboardInterrupt: 